In [1]:
import re
import requests
from datetime import datetime
import csv  # Import the csv module

# URL of the product
url = 'https://shopee.com.my/Kodak-ColorPlus-200-Color-Negative-Film-(35mm-Roll-Film)-(Exp-Date-01-2025)-i.64337744.20786894660?sp_atk=f5c793a2-7ecd-494d-8329-d37cd1e65a58&xptdk=f5c793a2-7ecd-494d-8329-d37cd1e65a58'

# Using regular expression to extract shop ID and item ID from the URL
r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

# Initialize offset for pagination
offset = 0

# Open CSV file in write mode and create a CSV writer object
with open('reviews.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Username', 'Date', 'Comment']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write header to the CSV file
    writer.writeheader()

    # Loop to fetch ratings until all ratings are fetched
    while True:
        ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
        # Fetch data from the ratings URL
        data = requests.get(ratings_url).json()

        # Loop through each rating in the fetched data
        for rating in data['data']['ratings']:
            # Extract username, comment, and date from the rating
            username = rating['author_username']
            comment = rating['comment']
            date = datetime.fromtimestamp(rating['ctime']).strftime('%Y-%m-%d %H:%M:%S')
            
            # Write data to the CSV file row by row
            writer.writerow({'Username': username, 'Date': date, 'Comment': comment})

        # Check if all ratings are fetched
        if len(data['data']['ratings']) < 20:
            break

        # Increment offset for pagination
        offset += 20